<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Agent/blob/main/smolagents/text_to_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installation
! pip install smolagents
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/smolagents.git

In [1]:
! pip install smolagents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 68.6 MB/s eta 0:00:00


# Text-to-SQL

In [2]:
!pip install sqlalchemy --upgrade -q

In this tutorial, we’ll see how to implement an agent that leverages SQL using `smolagents`.

> Let's start with the golden question: why not keep it simple and use a standard text-to-SQL pipeline?

A standard text-to-sql pipeline is brittle, since the generated SQL query can be incorrect. Even worse, the query could be incorrect, but not raise an error, instead giving some incorrect/useless outputs without raising an alarm.

👉 Instead, an agent system is able to critically inspect outputs and decide if the query needs to be changed or not, thus giving it a huge performance boost.

Let’s build this agent! 💪

Run the line below to install required dependencies:
```bash
!pip install smolagents python-dotenv sqlalchemy --upgrade -q
```
To call Inference Providers, you will need a valid token as your environment variable `HF_TOKEN`.
We use python-dotenv to load it.

In [3]:
from google.colab import userdata
import openai
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
openai.api_key  = os.environ["OPENAI_API_KEY"]

Then, we setup the SQL environment:

In [4]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    Float,
    insert,
    inspect,
    text,
)

engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

def insert_rows_into_table(rows, table, engine=engine):
    for row in rows:
        stmt = insert(table).values(**row)
        with engine.begin() as connection:
            connection.execute(stmt)

table_name = "receipts"
receipts = Table(
    table_name,
    metadata_obj,
    Column("receipt_id", Integer, primary_key=True),
    Column("customer_name", String(16), primary_key=True),
    Column("price", Float),
    Column("tip", Float),
)
metadata_obj.create_all(engine)

rows = [
    {"receipt_id": 1, "customer_name": "Alan Payne", "price": 12.06, "tip": 1.20},
    {"receipt_id": 2, "customer_name": "Alex Mason", "price": 23.86, "tip": 0.24},
    {"receipt_id": 3, "customer_name": "Woodrow Wilson", "price": 53.43, "tip": 5.43},
    {"receipt_id": 4, "customer_name": "Margaret James", "price": 21.11, "tip": 1.00},
]
insert_rows_into_table(rows, receipts)

### Build our agent

Now let’s make our SQL table retrievable by a tool.

The tool’s description attribute will be embedded in the LLM’s prompt by the agent system: it gives the LLM information about how to use the tool. This is where we want to describe the SQL table.

In [5]:
inspector = inspect(engine)
columns_info = [(col["name"], col["type"]) for col in inspector.get_columns("receipts")]

table_description = "Columns:\n" + "\n".join([f"  - {name}: {col_type}" for name, col_type in columns_info])
print(table_description)

Columns:
  - receipt_id: INTEGER
  - customer_name: VARCHAR(16)
  - price: FLOAT
  - tip: FLOAT


```text
Columns:
  - receipt_id: INTEGER
  - customer_name: VARCHAR(16)
  - price: FLOAT
  - tip: FLOAT
```

Now let’s build our tool. It needs the following: (read [the tool doc](https://huggingface.co/docs/smolagents/main/en/examples/../tutorials/tools) for more detail)
- A docstring with an `Args:` part listing arguments.
- Type hints on both inputs and output.

In [6]:
from smolagents import tool

@tool
def sql_engine(query: str) -> str:
    """
    Allows you to perform SQL queries on the table. Returns a string representation of the result.
    The table is named 'receipts'. Its description is as follows:
        Columns:
        - receipt_id: INTEGER
        - customer_name: VARCHAR(16)
        - price: FLOAT
        - tip: FLOAT

    Args:
        query: The query to perform. This should be correct SQL.
    """
    output = ""
    with engine.connect() as con:
        rows = con.execute(text(query))
        for row in rows:
            output += "\n" + str(row)
    return output

Now let us create an agent that leverages this tool.

We use the `CodeAgent`, which is smolagents’ main agent class: an agent that writes actions in code and can iterate on previous output according to the ReAct framework.

The model is the LLM that powers the agent system. `InferenceClientModel` allows you to call LLMs using HF’s Inference API, either via Serverless or Dedicated endpoint, but you could also use any proprietary API.

In [10]:
from smolagents import CodeAgent, InferenceClientModel

agent = CodeAgent(
    tools=[sql_engine],
    model=InferenceClientModel(model_id="meta-llama/Llama-3.3-70B-Instruct"), # meta-llama/Llama-3.1-8B-Instruct
)
agent.run("Can you give me the name of the client who got the most expensive receipt?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Can you give me the name of the client who got the most expensive receipt?                                      │
│                                                                                                                 │
╰─ InferenceClientModel - meta-llama/Llama-3.3-70B-Instruct ──────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  query = "SELECT customer_name FROM receipts ORDER BY price + tip DESC LIMIT 1"                                   
  result = sql_engine(query)                                                                                       
  print(result)                                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:

('Woodrow Wilson',)

Out: None

[Step 1: Duration 4.25 seconds| Input tokens: 2,079 | Output tokens: 97]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Woodrow Wilson")                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Woodrow Wilson

[Step 2: Duration 1.99 seconds| Input tokens: 4,347 | Output tokens: 146]

'Woodrow Wilson'

### Level 2: Table joins

Now let’s make it more challenging! We want our agent to handle joins across multiple tables.

So let’s make a second table recording the names of waiters for each receipt_id!

In [11]:
table_name = "waiters"
waiters = Table(
    table_name,
    metadata_obj,
    Column("receipt_id", Integer, primary_key=True),
    Column("waiter_name", String(16), primary_key=True),
)
metadata_obj.create_all(engine)

rows = [
    {"receipt_id": 1, "waiter_name": "Corey Johnson"},
    {"receipt_id": 2, "waiter_name": "Michael Watts"},
    {"receipt_id": 3, "waiter_name": "Michael Watts"},
    {"receipt_id": 4, "waiter_name": "Margaret James"},
]
insert_rows_into_table(rows, waiters)

Since we changed the table, we update the `SQLExecutorTool` with this table’s description to let the LLM properly leverage information from this table.

In [12]:
updated_description = """Allows you to perform SQL queries on the table. Beware that this tool's output is a string representation of the execution output.
It can use the following tables:"""

inspector = inspect(engine)
for table in ["receipts", "waiters"]:
    columns_info = [(col["name"], col["type"]) for col in inspector.get_columns(table)]

    table_description = f"Table '{table}':\n"

    table_description += "Columns:\n" + "\n".join([f"  - {name}: {col_type}" for name, col_type in columns_info])
    updated_description += "\n\n" + table_description

print(updated_description)

Allows you to perform SQL queries on the table. Beware that this tool's output is a string representation of the execution output.
It can use the following tables:

Table 'receipts':
Columns:
  - receipt_id: INTEGER
  - customer_name: VARCHAR(16)
  - price: FLOAT
  - tip: FLOAT

Table 'waiters':
Columns:
  - receipt_id: INTEGER
  - waiter_name: VARCHAR(16)


Since this request is a bit harder than the previous one, we’ll switch the LLM engine to use the more powerful [Qwen/Qwen2.5-Coder-32B-Instruct](https://huggingface.co/Qwen/Qwen2.5-Coder-32B-Instruct)!

In [13]:
sql_engine.description = updated_description

agent = CodeAgent(
    tools=[sql_engine],
    model=InferenceClientModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct"),  # Qwen/Qwen2.5-Coder-32B-Instruct
)

agent.run("Which waiter got more total money from tips?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Which waiter got more total money from tips?                                                                    │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Get unique waiter names                                                                                        
  unique_waiters_query = "SELECT DISTINCT waiter_name FROM waiters;"                                               
  unique_waiters_result = sql_engine(query=unique_waiters_query)                                                   
  print("Unique waiters:", unique_waiters_result)                                                                  
                                                                                                                   
  # Parse the unique waiter names                                                                                  
  unique_waiters = [name.strip() for name in unique_waiters_result.split('\n') if name.strip()]                    
                                                                                                                   
  # Get total tips for each waiter                                                                                 
  total_tips = {}                                                                                                  
  for waiter in unique_waiters:                                                                                    
      total_tips_query = f"""                                                                                      
      SELECT SUM(tip) AS totalTips FROM receipts WHERE receipt_id IN (                                             
          SELECT receipt_id FROM waiters WHERE waiter_name = '{waiter}'                                            
      );                                                                                                           
      """                                                                                                          
      total_tips_result = sql_engine(query=total_tips_query)                                                       
      total_tips[waiter] = float(total_tips_result.split('\n')[1].strip())                                         
      print(f"Total tips for {waiter}: {total_tips[waiter]}")                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Unique waiters: 
('Corey Johnson',)
('Michael Watts',)
('Margaret James',)

Code execution failed at line 'for waiter in unique_waiters:
    total_tips_query = f"""
    SELECT SUM(tip) AS totalTips FROM receipts WHERE receipt_id IN (
        SELECT receipt_id FROM waiters WHERE waiter_name = '{waiter}'
    );
    """
    total_tips_result = sql_engine(query=total_tips_query)
    total_tips[waiter\] = float(total_tips_result.split('\n')[1\].strip())
    print(f"Total tips for {waiter}: {total_tips[waiter\]}")' due to: OperationalError: (sqlite3.OperationalError) 
near "Corey": syntax error
[SQL: 
    SELECT SUM(tip) AS totalTips FROM receipts WHERE receipt_id IN (
        SELECT receipt_id FROM waiters WHERE waiter_name = '('Corey Johnson',)'
    );
    \]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Get unique waiter names                                                                                        
  unique_waiters_query = "SELECT DISTINCT waiter_name FROM waiters;"                                               
  unique_waiters_result = sql_engine(query=unique_waiters_query)                                                   
  print("Unique waiters result:", unique_waiters_result)                                                           
                                                                                                                   
  # Parse the unique waiter names correctly                                                                        
  unique_waiters = [line.strip().split("(")[1].split("'")[1] for line in unique_waiters_result.split('\n') if "("  
  in line]                                                                                                         
  print("Unique waiters:", unique_waiters)                                                                         
                                                                                                                   
  # Get total tips for each waiter                                                                                 
  total_tips = {}                                                                                                  
  for waiter in unique_waiters:                                                                                    
      total_tips_query = f"""                                                                                      
      SELECT SUM(tip) AS totalTips FROM receipts WHERE receipt_id IN (                                             
          SELECT receipt_id FROM waiters WHERE waiter_name = '{waiter}'                                            
      );                                                                                                           
      """                                                                                                          
      total_tips_result = sql_engine(query=total_tips_query)                                                       
      # Correct parsing of the result                                                                              
      if total_tips_result.strip() == '':                                                                          
          total_tips[waiter] = 0.0                                                                                 
      else:                                                                                                        
          # Replace comma with nothing to handle float conversion correctly                                        
          total_tips[waiter] = float(total_tips_result.split('\n')[1].strip().strip('()').replace(',', ''))        
      print(f"Total tips for {waiter}: {total_tips[waiter]}")                                                      
                                                                                                                   
  # Find the waiter with the most tips                                                                             
  max_tips_waiter = max(total_tips, key=total_tips.get)                                                            
  print("Waiter with the most tips:", max_tips_waiter)                                                             
  final_answer(max_tips_waiter)                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Unique waiters result: 
('Corey Johnson',)
('Michael Watts',)
('Margaret James',)
Unique waiters: ['Corey Johnson', 'Michael Watts', 'Margaret James']
Total tips for Corey Johnson: 1.2
Total tips for Michael Watts: 5.67
Total tips for Margaret James: 1.0
Waiter with the most tips: Michael Watts

Out - Final answer: Michael Watts

[Step 4: Duration 19.83 seconds| Input tokens: 13,578 | Output tokens: 1,342]

'Michael Watts'

It directly works! The setup was surprisingly simple, wasn’t it?

This example is done! We've touched upon these concepts:
- Building new tools.
- Updating a tool's description.
- Switching to a stronger LLM helps agent reasoning.

✅ Now you can go build this text-to-SQL system you’ve always dreamt of! ✨